In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("csv_data/bitcoin_1year_data.csv")

# Display the first 5 rows
print(df.head())

         date         price        volume
0  2024-04-23  66841.666852  2.774859e+10
1  2024-04-24  66406.952444  2.323223e+10
2  2024-04-25  64279.518129  3.093990e+10
3  2024-04-26  64486.183359  2.470535e+10
4  2024-04-27  63802.330049  2.330206e+10


In [2]:
# Show basic statistics for price and volume
summary = df[['price', 'volume']].describe()
print(summary)

               price        volume
count     365.000000  3.650000e+02
mean    76770.322150  4.166863e+10
std     15696.657107  3.110941e+10
min     53923.356111  6.340607e+09
25%     63128.224541  2.290854e+10
50%     69315.104123  3.288555e+10
75%     93663.447520  4.842692e+10
max    106182.236820  3.051904e+11


In [3]:
# Calculate percentage change in price and volume
df['price_change_pct'] = df['price'].pct_change() * 100
df['volume_change_pct'] = df['volume'].pct_change() * 100

# Display first few rows with the new columns
print(df[['date', 'price', 'volume', 'price_change_pct', 'volume_change_pct']].head(10))

         date         price        volume  price_change_pct  volume_change_pct
0  2024-04-23  66841.666852  2.774859e+10               NaN                NaN
1  2024-04-24  66406.952444  2.323223e+10         -0.650364         -16.275984
2  2024-04-25  64279.518129  3.093990e+10         -3.203632          33.176636
3  2024-04-26  64486.183359  2.470535e+10          0.321510         -20.150541
4  2024-04-27  63802.330049  2.330206e+10         -1.060465          -5.680100
5  2024-04-28  63517.248743  1.919048e+10         -0.446820         -17.644682
6  2024-04-29  63030.433617  1.618029e+10         -0.766430         -15.685841
7  2024-04-30  63797.679636  2.724298e+10          1.217263          68.371370
8  2024-05-01  60749.472092  3.953311e+10         -4.777929          45.113009
9  2024-05-02  58297.574075  5.110938e+10         -4.036081          29.282457


In [4]:
# Define default thresholds
price_threshold = 5     # percent
volume_threshold = 50   # percent

# Detect pump and dump days
df['is_pump'] = (df['price_change_pct'] > price_threshold) & (df['volume_change_pct'] > volume_threshold)
df['is_dump'] = (df['price_change_pct'] < -price_threshold) & (df['volume_change_pct'] > volume_threshold)

# Display detected days
print(df[df['is_pump'] | df['is_dump']][['date', 'price', 'volume', 'price_change_pct', 'volume_change_pct', 'is_pump', 'is_dump']])

           date         price        volume  price_change_pct  \
23   2024-05-16  66220.518314  3.810179e+10          7.554771   
28   2024-05-21  71430.297002  3.592348e+10          7.814902   
84   2024-07-16  64835.489733  3.781007e+10          6.388273   
105  2024-08-06  53956.261842  1.202460e+11         -6.981916   
123  2024-08-24  64049.685616  4.503867e+10          6.064405   
127  2024-08-28  59526.560880  3.813173e+10         -5.397149   
175  2024-10-15  66049.994927  4.655380e+10          5.125712   
198  2024-11-07  75620.886070  1.284925e+11          9.065295   
202  2024-11-11  80466.716548  9.347313e+10          5.006497   
217  2024-11-26  93004.700931  8.947632e+10         -5.112673   
308  2025-02-25  91396.766869  4.553970e+10         -5.118682   
314  2025-03-03  94261.532865  6.185911e+10          9.599735   
321  2025-03-10  80751.138933  3.201341e+10         -6.259180   
349  2025-04-07  78211.483582  3.614038e+10         -6.440989   
352  2025-04-10  82622.17

In [6]:
# Save the updated DataFrame to a new CSV for frontend/API use
df.to_csv("csv_data_processed/bitcoin_1year_data_with_flags.csv", index=False)

In [20]:
import pandas as pd
import plotly.graph_objects as go

# Read the processed CSV file
df = pd.read_csv("csv_data_processed/bitcoin_1year_data_with_flags.csv", parse_dates=["date"])

# Create traces for the price and volume
price_trace = go.Scatter(
    x=df['date'],
    y=df['price'],
    mode='lines',
    name='Price',
    line=dict(color='blue')
)

volume_trace = go.Scatter(
    x=df['date'],
    y=df['volume'],
    mode='lines',
    name='Volume',
    line=dict(color='green'),
    yaxis='y2'
)

# Create traces for pump and dump regions
pump_trace = go.Scatter(
    x=df[df['is_pump']]['date'],
    y=df[df['is_pump']]['price'],
    mode='markers',
    name='Pump',
    marker=dict(color='green', size=8, symbol='circle')
)

dump_trace = go.Scatter(
    x=df[df['is_dump']]['date'],
    y=df[df['is_dump']]['price'],
    mode='markers',
    name='Dump',
    marker=dict(color='red', size=8, symbol='circle')
)

# Create layout with dual y-axes
layout = go.Layout(
    title="Bitcoin Price and Volume with Pump and Dump Regions",
    xaxis=dict(title='Date'),
    yaxis=dict(title='Price', side='left'),
    yaxis2=dict(
        title='Volume',
        overlaying='y',
        side='right'
    ),
    showlegend=True
)

# Create the figure and add traces
fig = go.Figure(data=[price_trace, volume_trace, pump_trace, dump_trace], layout=layout)

# fig.update_layout(
#     plot_bgcolor='white',  # Change plot background
#     paper_bgcolor='lightgrey'  # Change overall figure background
# )

# Save the plot as an interactive HTML file
fig.write_html("bitcoin_price_volume_pump_dump_plot.html")

In [ ]:
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import plot

# Load processed data
df = pd.read_csv("csv_data_processed/bitcoin_1year_data_with_flags.csv", parse_dates=["date"])

# Price trace
price_trace = go.Scatter(
    x=df['date'],
    y=df['price'],
    mode='lines',
    name='Price',
    line=dict(color='blue'),
    visible=True  # default visible
)

# Volume trace
volume_trace = go.Scatter(
    x=df['date'],
    y=df['volume'],
    mode='lines',
    name='Volume',
    line=dict(color='green'),
    yaxis='y2',
    visible=False  # default hidden
)

# Layout with updatemenus (buttons)
layout = go.Layout(
    title='Bitcoin Interactive Graph: Price and Volume',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Price'),
    yaxis2=dict(
        title='Volume',
        overlaying='y',
        side='right',
        showgrid=False
    ),
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            x=0.5,
            y=1.2,
            showactive=True,
            buttons=list([
                dict(label="Price Only",
                     method="update",
                     args=[{"visible": [True, False]},
                           {"title": "Bitcoin Price Only"}]),
                dict(label="Volume Only",
                     method="update",
                     args=[{"visible": [False, True]},
                           {"title": "Bitcoin Volume Only"}]),
                dict(label="Price + Volume",
                     method="update",
                     args=[{"visible": [True, True]},
                        {"title": "Bitcoin Price + Volume"}])
            ]),
        )
    ]
)

# Create and save the figure
fig = go.Figure(data=[price_trace, volume_trace], layout=layout)
plot(fig, filename='interactive_price_volume_toggle.html', auto_open=False)


'interactive_price_volume_toggle.html'

In [33]:
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import plot

# Load processed data
df = pd.read_csv("csv_data_processed/bitcoin_1year_data_with_flags.csv", parse_dates=["date"])

# Traces
price_trace = go.Scatter(
    x=df['date'],
    y=df['price'],
    mode='lines',
    name='Price',
    line=dict(color='deepskyblue'),
    visible=True
)

volume_trace = go.Scatter(
    x=df['date'],
    y=df['volume'],
    mode='lines',
    name='Volume',
    line=dict(color='limegreen'),
    yaxis='y2',
    visible=False
)

# Pump markers (green dots)
pump_trace = go.Scatter(
    x=df[df['is_pump'] == True]['date'],
    y=df[df['is_pump'] == True]['price'],
    mode='markers',
    name='Pump',
    marker=dict(color='lime', size=10, symbol='circle'),
    visible=True
)

# Dump markers (red dots)
dump_trace = go.Scatter(
    x=df[df['is_dump'] == True]['date'],
    y=df[df['is_dump'] == True]['price'],
    mode='markers',
    name='Dump',
    marker=dict(color='red', size=10, symbol='x'),
    visible=True
)

# Layout with dual y-axes and custom styling
layout = go.Layout(
    title='Bitcoin Price and Volume with Pump/Dump Markers',
    xaxis=dict(title='Date', color='white'),
    yaxis=dict(title='Price', color='deepskyblue'),
    yaxis2=dict(
        title='Volume',
        overlaying='y',
        side='right',
        showgrid=False,
        color='limegreen'
    ),
    plot_bgcolor='#0f0f23',  # Dark background
    paper_bgcolor='#0f0f23',
    font=dict(color='white'),
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            x=0.5,
            y=1.2,
            showactive=True,
            buttons=list([
                dict(label="Price Only",
                     method="update",
                     args=[{"visible": [True, False, True, True]},
                           {"title": "Bitcoin Price Only"}]),
                dict(label="Volume Only",
                     method="update",
                     args=[{"visible": [False, True, False, False]},
                           {"title": "Bitcoin Volume Only"}]),
                dict(label="Price + Volume",
                     method="update",
                     args=[{"visible": [True, True, True, True]},
                           {"title": "Bitcoin Price + Volume"}])
            ]),
        )
    ]
)

# Create figure
fig = go.Figure(data=[price_trace, volume_trace, pump_trace, dump_trace], layout=layout)
plot(fig, filename='interactive_price_volume_pump_dump.html', auto_open=False)

'interactive_price_volume_pump_dump.html'

In [3]:
import pandas as pd
import plotly.graph_objs as go

# Load processed data
file_path = "csv_data_processed/bitcoin_1year_data_with_flags.csv"
df = pd.read_csv(file_path)
df['date'] = pd.to_datetime(df['date'])

# Create base traces
price_trace = go.Scatter(x=[], y=[], mode='lines', name='Price', line=dict(color='orange'))
volume_trace = go.Scatter(x=[], y=[], mode='lines', name='Volume', yaxis='y2', line=dict(color='blue'))
pump_trace = go.Scatter(x=[], y=[], mode='markers', name='Pump', marker=dict(color='green', size=10))
dump_trace = go.Scatter(x=[], y=[], mode='markers', name='Dump', marker=dict(color='red', size=10))

# Create animation frames
frames = []
for i in range(1, len(df)):
    frame = go.Frame(
        data=[
            go.Scatter(x=df['date'][:i], y=df['price'][:i], mode='lines', line=dict(color='orange')),
            go.Scatter(x=df['date'][:i], y=df['volume'][:i], mode='lines', yaxis='y2', line=dict(color='blue')),
            go.Scatter(x=df['date'][:i][df['is_pump'][:i]], y=df['price'][:i][df['is_pump'][:i]], mode='markers', marker=dict(color='green', size=10)),
            go.Scatter(x=df['date'][:i][df['is_dump'][:i]], y=df['price'][:i][df['is_dump'][:i]], mode='markers', marker=dict(color='red', size=10)),
        ],
        name=str(i)
    )
    frames.append(frame)

# Define layout with animation controls
layout = go.Layout(
    title="Animated Price and Volume with Pump/Dump Markers",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Price"),
    yaxis2=dict(title="Volume", overlaying='y', side='right'),
    plot_bgcolor='rgb(250, 250, 255)',
    updatemenus=[{
        "type": "buttons",
        "buttons": [
            {"label": "Play", "method": "animate", "args": [None, {"frame": {"duration": 50, "redraw": True}, "fromcurrent": True}]},
            {"label": "Pause", "method": "animate", "args": [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]}
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }],
    sliders=[{
        "steps": [
            {"args": [[f.name], {"frame": {"duration": 50, "redraw": True}, "mode": "immediate"}], "label": str(i), "method": "animate"}
            for i, f in enumerate(frames)
        ],
        "transition": {"duration": 0},
        "x": 0,
        "y": -0.1,
        "currentvalue": {"prefix": "Day: "}
    }]
)

# Combine figure and save as HTML
fig = go.Figure(data=[price_trace, volume_trace, pump_trace, dump_trace], layout=layout, frames=frames)
fig.write_html("Testing/animated_bitcoin_price_volume.html")
